In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./data/Joined_table.csv", sep='\t')

In [4]:
# list of regions
regions = ['Вінницька область', 'Волинська область','Дніпропетровська область']

### Count candidates by party and council

In [5]:
def get_candidates_stat_wide(df, region, elected = False):
    '''
    Function to get pivot table of number of candidates/elected candidates from parties in city councils.
    City councils as columns.
    '''
    obl = region
    header = 'Кількість кандидатів'
    if elected:
        wide_df = pd.pivot_table(df[df['Область'] == obl], index="Партія", columns="Міська рада", values='Статус', aggfunc=len, fill_value=0)
    else:
        wide_df = pd.pivot_table(df[df['Область'] == obl], index="Партія", columns="Міська рада", values='Статус', aggfunc=lambda x:(x=='обрано').sum(), fill_value=0)
        header = 'Кількість обраних кандидатів'
    wide_df.columns = pd.MultiIndex.from_tuples(zip([header]*len(wide_df.columns), wide_df.columns))
    return wide_df

In [6]:
def get_candidates_stat_long(df, region):
    '''
    Function to get pivot table of number of candidates from parties in city councils.
    Election status as columns.
    '''
    df = df.fillna(0)
    long_df = pd.pivot_table(df[['Партія', 'Область', 'Статус', 'Міська рада']][df['Область'] == region], index=['Партія', 'Міська рада'], columns='Статус', aggfunc=len, margins=True)
    long_df.columns = long_df.columns.droplevel(0)
    col_names = ['Кандидати', 'Обрано', 'Не обрано']
    long_df = long_df.rename(columns={'не обрано': 'Не обрано', 'обрано': 'Обрано', 'All': 'Кандидати'}).reindex(columns = col_names)
    return long_df.fillna(0).astype(int)

In [7]:
vinn_cand, vol_cand, dnipr_cand = [get_candidates_stat_long(df, i) for i in regions]

### Save output

In [8]:
vinn_cand.to_csv('./data/Vinnitsa_data.csv')
vol_cand.to_csv('./data/Volyn_data.csv')
dnipr_cand.to_csv('./data/Dnipro_data.csv')